# Upload the dataset to Argilla

Before we can upload the dataset to Argilla, we need to apply some transformations and postprocessing.

## Load and transform the dataset

First, we load the dataset.


In [29]:

from datasets import load_dataset
import pandas as pd
import os
dataset = load_dataset("data-is-better-together/image-preferences")
df = dataset.to_pandas()
df

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

,quality_prompt,category,subcategory,style_prompt,simplified_prompt,__index_level_0__,grouped_model_name,prompt,distilabel_metadata,image_quality_dev,image_simplified_dev,image_quality_sd,image_simplified_sd
0,"a harp without strings, in an anime style, wit...",Anime,anime style,"a harp without strings, in an anime style, wit...","A harp without strings, in anime style, with i...",1,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,a harp without any strings,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1,A majestic horse perched on an astronaut's sho...,Cinematic,bokeh,"A horse perched on an astronaut's shoulders, c...","A horse on an astronaut's shoulders, with cine...",2,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,A horse sitting on an astronaut's shoulders.,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
2,"a 3D model of a minimalist, abstract financial...",3D Model,volumetric,"a 3D model of a minimalist, abstract financial...",3D model of an abstract financial landscape wi...,4,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,a real photography that is minamalistic and th...,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
3,"Tempera painting of a fragmented, distorted sc...",Painting,Tempera,Tempera painting of a fragmented scene from th...,"Tempera painting with distorted, blurred fragm...",15,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,"digital art in the style of epic Surreal, abst...",{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
4,"a crown, captured in sharp black and white wit...",Photographic,Fomapan,"a crown, captured in sharp black and white, wi...","A sharp, black and white crown with high contr...",16,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,a crown,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,"Cristiano Ronaldo plays chess with Shrek, clay...",Animation,Claymation,"Cristiano Ronaldo Plays Chess with Shrek, clay...",Cristiano Ronaldo plays chess with Shrek in a ...,14594,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,"Cristiano Ronaldo Plays Chess with Shrek, intr...",{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
8785,"a highly detailed red panda, ultra-sharp focus...",Photographic,highly detailed,"a highly detailed red panda, ultra-sharp focus...",A detailed red panda with vibrant colors in st...,14595,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,a cute red panda high qualtity,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
8786,"Pop Art: Bold, vibrant ship sails serene, icy ...",Painting,Pop Art,"Pop Art: Bold, vibrant ship 

Now, we transform the dataset to match the format required by Argilla.

In [30]:
prompt_columns = df.columns.where(df.columns.str.endswith("_prompt")).dropna()
df["generation"] = df[prompt_columns].apply(lambda row: row.values, axis=1)
df = df.drop(columns=prompt_columns)
df

,category,subcategory,__index_level_0__,grouped_model_name,prompt,distilabel_metadata,image_quality_dev,image_simplified_dev,image_quality_sd,image_simplified_sd,generation
0,Anime,anime style,1,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,a harp without any strings,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[a harp without strings, in an anime style, wi..."
1,Cinematic,bokeh,2,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,A horse sitting on an astronaut's shoulders.,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[A majestic horse perched on an astronaut's sh...
2,3D Model,volumetric,4,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,a real photography that is minamalistic and th...,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[a 3D model of a minimalist, abstract financia..."
3,Painting,Tempera,15,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,"digital art in the style of epic Surreal, abst...",{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[Tempera painting of a fragmented, distorted s..."
4,Photographic,Fomapan,16,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,a crown,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[a crown, captured in sharp black and white wi..."
...,...,...,...,...,...,...,...,...,...,...,...
8784,Animation,Claymation,14594,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,"Cristiano Ronaldo Plays Chess with Shrek, intr...",{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[Cristiano Ronaldo plays chess with Shrek, cla..."
8785,Photographic,highly detailed,14595,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,a cute red panda high qualtity,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[a highly detailed red panda, ultra-sharp focu..."
8786,Painting,Pop Art,14596,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,"An epic, serene oil painting: Majestic ship sa...",{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"[Pop Art: Bold, vibrant ship sails serene, icy..."
8787,Animation,Vector,14597,[https://f94i5ss7a040r0v5.us-east-1.aws.endpoi...,Nice photo about primitive people in the forest,{'raw_input_image_gen_quality_dev': {'prompt':...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,[Stylized vector animation of primitive people...


We also extract the categories and subcategories from the dataset. These will be used as metadata properties in Argilla.

In [52]:
categories = list(df_filtered.category.unique())
sub_categories = list(df_filtered.subcategory.unique())
categories, sub_categories


(['Anime',
  'Cinematic',
  '3D Model',
  'Painting',
  'Photographic',
  'Illustration',
  'Manga',
  'Fantasy art',
  'Digital art',
  'Pixel art',
  'Neonpunk',
  'Animation'],
 ['anime style',
  'bokeh',
  'volumetric',
  'Tempera',
  'Fomapan',
  'grainy',
  'dramatic lighting',
  'moody',
  'Fantasy',
  'Digital',
  'Ink Wash',
  'high-energy',
  'Comics',
  'majestic',
  'iconic',
  'painterly',
  '8-bit graphics',
  'key visual',
  'epic',
  'detailed',
  'Sketch',
  'highly detailed',
  'Ektar',
  'dark purple shadows',
  'pixel art style',
  'dreamy',
  'octane render',
  'ultramodern',
  'Renaissance',
  'ethereal',
  'Landscape',
  'blocky',
  'low-res',
  'Pop Art',
  'Anime',
  'magical',
  'vibrant',
  'Hand-drawn',
  'film grain',
  'Pixel Art',
  'Oil',
  'Animation',
  'Mural',
  'Watercolor',
  'Impressionism',
  'studio anime',
  'Expressionism',
  'Vector',
  'CineStill 50D',
  'ColorPlus',
  '4k',
  'magenta highlights',
  'Gold',
  'Realism',
  'illustrative',
  


## Upload the dataset to Argilla

We will use the Argilla Python client to upload the dataset to Argilla. We assume you are familiar with Argilla and how to deploy it. If not, please refer to the [Argilla quickstart documentation](https://docs.argilla.io/dev/getting_started/quickstart/).

In [71]:
import argilla as rg
import os

client = rg.Argilla(
    api_url="https://data-is-better-together-image-preferences.hf.space",
)


Let's first generate some embeddings for the prompts.

In [53]:
from model2vec import StaticModel

model_name = "minishlab/potion-base-8M"
model = StaticModel.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/30.2M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/271k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/684k [00:00<?, ?B/s]

In [61]:
generations = [gen[0] for gen in df["generation"]]
df["prompt_embedding"] = model.encode(generations, max_length=512).tolist()



/var/folders/9t/msy700h16jz3q35qvg4z1ln40000gn/T/ipykernel_81166/298091170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["prompt_embedding"] = model.encode(generations, max_length=512).tolist()


We will now define the settings for the dataset. We will use a custom template for the representation of the images and a label question for the preference. Additionally, we will define the metadata properties for the dataset.

In [93]:
settings = rg.Settings(
    fields=[
        rg.CustomField("images", template="template.html"),
    ],
    questions=[
        rg.LabelQuestion(
            name="preference",
            title="Image preference",
            description="Which one is the overall better image (prompt adherence, semantics, and aesthetics)?",
            labels={"image_1": "Image 1 is better", "image_2": "Image 2 is better", "both_good": "Both images are good", "both_bad": "Both images are bad", "toxic_content": "Toxic content ⚠️"},
        ),
    ],
    metadata=[
        rg.TermsMetadataProperty(
            name="model_1",
            options=[
                "dev",
                "sd",
            ]
        ),
        rg.TermsMetadataProperty(
            name="model_2",
            options=[
                "dev",
                "sd",
            ]
        ),
        rg.TermsMetadataProperty(
            name="evolution",
            options=[
                "simplified",
                "quality"            
            ]
        ),
    ],
    vectors=[rg.VectorField(name="prompt", dimensions=256)],
    allow_extra_metadata=True,
)
dataset = rg.Dataset(
    name="image_preferences",
    settings=settings,
)
dataset.create()

/Users/davidberenstein/Library/Caches/pypoetry/virtualenvs/api-nlp-wikification-weaviate-ZdxY32E0-py3.10/lib/python3.10/site-packages/argilla/datasets/_resource.py:264: UserWarning: Workspace not provided. Using default workspace: argilla id: 3486a24c-0ec7-4f8a-b0c2-b721d9b02484
  warnings.warn(f"Workspace not provided. Using default workspace: {workspace.name} id: {workspace.id}")


Dataset(id=UUID('f3ad715c-5a03-4193-8ac0-c968d8a40aab') inserted_at=datetime.datetime(2024, 11, 25, 21, 29, 25, 650044) updated_at=datetime.datetime(2024, 11, 25, 21, 29, 27, 273793) name='image_preferences' status='ready' guidelines=None allow_extra_metadata=True distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('3486a24c-0ec7-4f8a-b0c2-b721d9b02484') last_activity_at=datetime.datetime(2024, 11, 25, 21, 29, 27, 273793))

In [54]:
evolution

'simplified'

In [94]:
from importlib import metadata
import requests
from io import BytesIO
import random
import base64
from PIL import Image

dataset_name = "data-is-better-together/image-preferences-filtered"
output_dir = "images"


def make_image_from_url(idx, image_column):
    base_url = f"https://huggingface.co/datasets/{dataset_name}/resolve/main"
    full_url = "/".join([base_url, image_column, f"{idx}.jpg"])
    return full_url

def make_image_from_bytes(bytes_data: bytes):
    base64_data = base64.b64encode(bytes_data).decode('utf-8')
    return base64_data

records = []
for i, row in df.iterrows():
    for source, generation in zip(prompt_columns, row["generation"]):
        evolution = source.split("_")[0]
        image_columns = df.columns.where(
            df.columns.str.startswith(f"image_{evolution}")
        ).dropna()
        if not image_columns.empty:
            image_columns = image_columns.tolist()
            random.shuffle(image_columns)   
            image_1 = row["__index_level_0__"]
            image_2 = row["__index_level_0__"]
            image_1 = make_image_from_url(image_1, image_columns[0])
            image_2 = make_image_from_url(image_2, image_columns[1])
            # print(image_1, image_2)
            # image_1 = make_image_from_bytes(row[image_columns[0]]["bytes"])
            # image_2 = make_image_from_bytes(row[image_columns[1]]["bytes"])
            model_1 = image_columns[0].split("_")[-1]
            model_2 = image_columns[1].split("_")[-1]

            record = rg.Record(
                id=f"{row['__index_level_0__']}-{evolution}",
                fields={
                    "images": {
                        "image_1": image_1,
                        "image_2": image_2,
                        "prompt": generation,
                    }
                },
                metadata={
                    "model_1": model_1,
                    "model_2": model_2,
                    "evolution": evolution,
                    "category": row.category,
                    "sub_category": row.subcategory,
                },
                vectors={
                    "prompt": row["prompt_embedding"],
                }
            )
            records.append(record)

In [95]:
first_5000 = records[:5000]
random.shuffle(first_5000)

In [96]:
client.datasets(name="image_preferences").records.log(first_5000)


Sending records...: 20batch [00:45,  2.29s/batch]                     


DatasetRecords(Dataset(id=UUID('f3ad715c-5a03-4193-8ac0-c968d8a40aab') inserted_at=datetime.datetime(2024, 11, 25, 21, 29, 25, 650044) updated_at=datetime.datetime(2024, 11, 25, 21, 29, 27, 273793) name='image_preferences' status='ready' guidelines=None allow_extra_metadata=True distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('3486a24c-0ec7-4f8a-b0c2-b721d9b02484') last_activity_at=datetime.datetime(2024, 11, 25, 21, 29, 27, 273793)))